##Day 2 Hands-On Lab:
Mastering Prompt Engineering: This notebook covers the practical application of advanced Prompt Engineering frameworks like R.O.L.E.S. and Chain of Thought (CoT).

We will compare the results of poorly-structured prompts against engineered prompts using LangChain and two powerful models: OpenAI (GPT-4o) and Gemini model

Setup & Environment Configuration: We need to install the necessary libraries and set up our API keys.

We recommend using Colab Secrets for secure storage of your OPENAI_API_KEY and GEMINI_API_KEY.#

Install LangChain and necessary dependencies


In [ ]:
!pip install langchain==0.3.7 langchain-core==0.3.15 langchain-google-genai==2.0.4 google-generativeai==0.8.3 protobuf==5.27.0

  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata



In [ ]:

# --- API Key Setup ---
# Load environment variables (assumes using Colab Secrets or a .env file)
# If using Colab Secrets, click the key icon on the left panel.
# Variables must be named OPENAI_API_KEY and GEMINI_API_KEY.

# Get API keys from environment variables

gemini_api_key = userdata.get("GOOGLE_API_KEY")
if not gemini_api_key:
    print("Warning: GEMINI_API_KEY not found. Gemini model will not run.")

# openai_api_key = userdata.get("OPENAI_API_KEY")
# if not openai_api_key:
#     print("Warning: OPENAI_API_KEY not found. OpenAI model will not run.")


In [ ]:


# --- Model Initialization ---
# 1. OpenAI Model
# Using a powerful model like gpt-4o for complex tasks
# openai_model = ChatOpenAI(
#     model="gpt-4o-mini",
#     api_key=openai_api_key,
#     temperature=0.3
# )

# 2. Google Gemini Model
# Using gemini-2.5-flash for a highly performant and cost-effective alternative
gemini_model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite-001",
    google_api_key=gemini_api_key,
    temperature=0.3
)

print("Setup complete. Models initialized.")

Setup complete. Models initialized.




2. Advanced Reasoning: Chain of Thought (CoT) The CoT technique involves adding the phrase "Let's think step by step" to the prompt. This forces the LLM to structure its reasoning before providing a final answer, dramatically improving accuracy in logic and mathematical problems.



In [ ]:
user_query = """
A farmer has 17 goats. All but 9 run away. How many goats are left?
"""

# --- Standard Prompt  ---
standard_prompt = ChatPromptTemplate.from_messages([
    ("user", "{query}")
])

chain_standard = standard_prompt | gemini_model | StrOutputParser()
print("--- GPT Standard Response ---")
print(chain_standard.invoke({"query": user_query}))



--- GPT Standard Response ---
If all but 9 goats run away, that means 9 goats are left.



In [ ]:

# --- Chain of Thought (CoT) Prompt (Reasoning) ---
cot_prompt = ChatPromptTemplate.from_messages([
    ("user", "Let's think step by step to find the correct answer. {query}")
])
chain_cot = cot_prompt | gemini_model | StrOutputParser()
print("\n--- GPT-4o CoT Response (Correct) ---")
print(chain_cot.invoke({"query": user_query}))


--- GPT-4o CoT Response (Correct) ---
Here's how to solve the problem:

*   The problem states "All but 9 run away."
*   This means 9 goats did *not* run away.

Answer: 9



**3. The R.O.L.E.S. Prompt Engineering Framework : **

R.O.L.E.S. ensures every critical aspect of the desired output is explicitly defined, reducing ambiguity and improving consistency.

Role: The identity the LLM adopts (e.g., expert, beginner).Sets expertise,tone, and knowledge boundaries.

Objective: The core task to accomplish (e.g., summarize, critique, generate).Defines the purpose of the output.

Limitations: Constraints on length, format, or content (e.g., max 50 words, exclude jargon).Ensures practical, usable results.

Examples: Few-shot learning examples (Input $\to$ Output pairs).Guides model on specific required style or format.

Style: Tone, language complexity, and output format (e.g., professional, JSON, Markdown).


## Example 1 : Write a blog post

In [ ]:
def generate_roles_prompt(role, objective, limitations, style, query):
    """Generates a structured prompt based on the R.O.L.E.S. framework."""
    prompt_template = f"""
    --- INSTRUCTIONS (R.O.L.E.S. FRAMEWORK) ---
    ROLE: {role}
    OBJECTIVE: {objective}
    LIMITATIONS: {limitations}
    STYLE: {style}
    --- USER QUERY ---
    {query}
    """
    return prompt_template

# Define a simple chain template for easy switching between models
def create_roles_chain(model):
    prompt = ChatPromptTemplate.from_template("{input}")
    return prompt | model | StrOutputParser()


blog_prompt = "Write a blog post about AI."
print("---  PROMPT (Gemini) ---")
print(create_roles_chain(gemini_model).invoke({"input": blog_prompt}))


---  PROMPT (Gemini) ---
## The Robots are Coming (and They're Already Here): A Look at the Ever-Evolving World of AI

For years, we've been bombarded with headlines about Artificial Intelligence. From self-driving cars to algorithms that recommend our next binge-worthy show, AI has steadily crept into our lives, transforming the way we work, play, and interact with the world. But what exactly *is* AI, and what does the future hold for this rapidly evolving technology?

**Decoding the Buzzwords: What is AI, Really?**

Let's start with the basics. Artificial Intelligence, at its core, refers to the ability of a computer or a robot controlled by a computer to perform tasks that typically require human intelligence. This can range from simple tasks like recognizing patterns to complex ones like making decisions, learning from experience, and even creating art.

Think of it like this:

*   **Narrow or Weak AI:** This is the type of AI we see most often today. It's designed to perform a spe

In [ ]:


# --- R.O.L.E.S. ENGINEERED PROMPT ---
roles_blog_prompt = generate_roles_prompt(
    role="A seasoned Marketing Director and AI expert.",
    objective="Write a compelling, 300-word introduction to the R.O.L.E.S. prompt engineering framework.",
    limitations="Maximum 300 words. Must not use the word 'chatbot' or 'AI helper'.",
    style="Enthusiastic, engaging, and structured as Markdown with a H2 title.",
    query="Write the post now."
)

print("\n--- ENGINEERED PROMPT (Gemini 2.5 Flash) ---")
print(create_roles_chain(gemini_model).invoke({"input": roles_blog_prompt}))


--- ENGINEERED PROMPT (Gemini 2.5 Flash) ---
## Unlock the Power of Language Models: Introducing R.O.L.E.S.

Alright, marketing mavens and tech enthusiasts! Are you ready to supercharge your content creation, streamline your campaigns, and unlock previously unimaginable levels of efficiency? Then buckle up, because we're about to dive into the revolutionary world of prompt engineering – and I'm thrilled to introduce you to a framework that will change the way you interact with language models forever: **R.O.L.E.S.**

Forget vague instructions and frustrating results. R.O.L.E.S. provides a structured approach to crafting prompts that elicit *exactly* what you need. Think of it as a secret weapon, a cheat code, a finely tuned instrument for extracting brilliance from these powerful engines.

**R.O.L.E.S.** stands for:

*   **R**ole: Define the persona you want the model to adopt (e.g., a seasoned copywriter, a witty social media guru).
*   **O**bjective: Clearly state the desired outcom

#Prompt Set 2: Code Review

In [ ]:
code_snippet = "def sum_list(l): total = 0; for i in l: total += i; return total"


# ---  PROMPT TEST ---
code_prompt = f"Fix this code and tell me what is wrong: {code_snippet}"
print("--- PROMPT (Gemini 2.5 Flash) ---")
print(create_roles_chain(gemini_model).invoke({"input": code_prompt}))


# --- R.O.L.E.S. ENGINEERED PROMPT ---
roles_code_prompt = generate_roles_prompt(
    role="A Senior Python Developer specializing in clean, efficient code.",
    objective="Refactor the provided code snippet for efficiency (using built-in methods), add a docstring, and identify a single major bug in the logic (if any).",
    limitations="Do not change the function name. Output the revised code first, then provide a single paragraph critique.",
    style="Formal tone. Output must be in two sections: 'Revised Code' and 'Critique'.",
    query=f"Here is the code: {code_snippet}"
)

print("\n--- ENGINEERED PROMPT (Gemini) ---")
print(create_roles_chain(gemini_model).invoke({"input": roles_code_prompt}))

--- PROMPT (Gemini 2.5 Flash) ---
```python
def sum_list(l):
  """
  Calculates the sum of all numbers in a list.

  Args:
    l: A list of numbers.

  Returns:
    The sum of the numbers in the list.
  """
  total = 0
  for i in l:
    total += i
  return total
```

**What was wrong (and the fixes):**

The original code was perfectly functional and correct.  There was nothing inherently *wrong* with it.  However, I've added:

*   **Docstring:**  A docstring (the text within triple quotes `"""..."""`) is crucial for good code.  It explains what the function does, what arguments it takes, and what it returns.  This makes the code much easier to understand and use.  It's good practice.
*   **Comments (Optional):**  While the code is simple enough that comments aren't strictly *needed*, in more complex functions, comments can clarify the logic.

The core logic of the original code (initializing `total` to 0, iterating through the list, and adding each element to `total`) is correct and ef

# Prompt Set 3: Data Analysis


In [ ]:
roles_finance_prompt = generate_roles_prompt(
    role="A highly skeptical Senior Financial Analyst.",
    objective="Analyze the past week's fictional performance of AAPL stock based on technical analysis, and make a plausible, short-term prediction.",
    limitations="Do not use real-time search. Analysis must be a single paragraph.",
    style="Highly technical, formal, and use the term 'stochastic' at least once.",
    query="Perform the analysis."
)

print("\n--- ENGINEERED PROMPT (Gemini 2.5 Flash) ---")
print(create_roles_chain(gemini_model).invoke({"input": roles_finance_prompt}))


--- ENGINEERED PROMPT (Gemini 2.5 Flash) ---
Alright, let's dissect this week's AAPL charade. The stock exhibited a rather uninspiring sideways drift, failing to convincingly break either the 170 or 175 resistance levels. Volume remained tepid throughout the period, suggesting a lack of conviction from either bulls or bears. The 50-day moving average continues to act as a mild support, but the Relative Strength Index (RSI) hovers around a neutral 55, indicating a lack of strong momentum. Furthermore, the stochastic oscillator is currently showing a potential bearish crossover, which, coupled with the stagnant price action, hints at a possible short-term pullback towards the 168 level. I maintain a cautious outlook; the market appears indecisive, and any significant move will likely require a catalyst we haven't yet seen.



# Prompt Set 4: Summarization


In [ ]:

long_text = """
The company, Acme Corp, announced its Q3 earnings on October 15th, reporting a staggering
revenue increase of $45 million, surpassing last year's figure of $30 million.
The CEO stated that hiring will increase by 20% in the new year, starting January 1st.
This growth is primarily attributed to their successful digital marketing campaign launched in July.
The stock reacted positively, climbing $5 per share within 48 hours of the announcement.
"""

# ---  PROMPT TEST ---
bad_summarize_prompt = f"Summarize this paragraph: {long_text}"
print("--- NORMAL PROMPT (Gemini model) ---")
print(create_roles_chain(gemini_model).invoke({"input": bad_summarize_prompt}))


# --- R.O.L.E.S. ENGINEERED PROMPT ---
roles_summarize_prompt = generate_roles_prompt(
    role="A News Editor reviewing a financial wire report.",
    objective="Summarize the text, focusing only on reported dates and financial figures.",
    limitations="Must be exactly 2 sentences long. Include no opinion or analysis.",
    style="Objective and journalistic.",
    query=f"Summarize this text: {long_text}"
)

print("\n--- ENGINEERED PROMPT (Gemini 2.5 Flash) ---")
print(create_roles_chain(gemini_model).invoke({"input": roles_summarize_prompt}))

--- NORMAL PROMPT (Gemini model) ---
Acme Corp. announced strong Q3 earnings on October 15th, with revenue increasing by $45 million, exceeding last year's. The company attributes this growth to a successful digital marketing campaign and plans to increase hiring by 20% starting January 1st. The positive news led to a $5 per share increase in stock price within two days.


--- ENGINEERED PROMPT (Gemini 2.5 Flash) ---
Acme Corp reported Q3 earnings on October 15th, with revenue increasing to $45 million, exceeding the previous year's $30 million. Hiring is projected to increase by 20% starting January 1st.



Prompt Set 5: Topic Classification




In [ ]:

classification_text = "The latest driver updates fixed the latency issues on the GPU, improving frame rates substantially."

# --- NORMAL/BAD PROMPT TEST ---
bad_classify_prompt = f"What is this text about: {classification_text}"
print("--- BAD PROMPT (Gemini 2.5 Flash) ---")
print(create_roles_chain(gemini_model).invoke({"input": bad_classify_prompt}))


# --- R.O.L.E.S. ENGINEERED PROMPT (Using Examples for format) ---
roles_classify_prompt = generate_roles_prompt(
    role="A Data Scientist performing classification.",
    objective="Classify the following text into one of these categories: [Hardware, Software, Finance].",
    limitations="Output must be a single word and case-sensitive (e.g., 'Hardware').",
    style="Raw text output. Use the following example:",
    query=f"""
    Example Input: "The market closed high."
    Example Output: Finance

    Classify this input: "{classification_text}"
    """
)

print("\n--- ENGINEERED PROMPT (GPT-4o) ---")
print(create_roles_chain(openai_model).invoke({"input": roles_classify_prompt}))

--- BAD PROMPT (Gemini 2.5 Flash) ---
This text is about **a positive change in the performance of a computer's graphics card (GPU)**. Specifically:

*   **Driver updates:** Software updates for the GPU.
*   **Fixed latency issues:** Resolved delays or lag in processing data.
*   **Improved frame rates substantially:** Made the game or application run smoother and faster, leading to a better visual experience.

In short, the text describes how updating the GPU's drivers has improved its performance.

--- ENGINEERED PROMPT (GPT-4o) ---
Hardware


Prompt Set 6: Sentiment Analysis

Determine if the customer sentiment is positive, neutral, or negative.

Output must be a JSON object with keys sentiment and confidence (1-100).

In [ ]:
sentiment_review = "The product works exactly as advertised, but the delivery took far too long, making the overall experience frustrating."


# --- R.O.L.E.S. ENGINEERED PROMPT ---
roles_sentiment_prompt = generate_roles_prompt(
    role="A Customer Service Manager focused on customer feedback.",
    objective="Analyze the provided review text to determine overall customer sentiment (Positive, Negative, or Neutral).",
    limitations="Only use the exact labels: 'Positive', 'Negative', or 'Neutral'.",
    style="Strictly output a JSON object.",
    query=f"""
    Analyze this review: "{sentiment_review}"

    Required JSON format:
    {{
      "sentiment": "<LABEL>",
      "confidence": "<1-100 score>"
    }}
    """
)

print("\n--- ENGINEERED PROMPT (GPT-4o) ---")
print(create_roles_chain(openai_model).invoke({"input": roles_sentiment_prompt}))



--- ENGINEERED PROMPT (GPT-4o) ---
{
  "sentiment": "Negative",
  "confidence": 85
}
